# future_emis
## Purpose: input IAM projections of future emissions under different policy scenarios
- Data from NGFS

### 01/29/24, Erica Simon

In [29]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pooch
import copy

from fair import FAIR
from fair.interface import fill, initialise
from fair.io import read_properties

## Import historical emissions dataset 
`hist_emis`

In [30]:
hist_emis = pd.read_csv('~/inputs/all_scaled_1750-2022.csv')

# remove 'Emissions|' from variable name
for i in range(len(hist_emis.variable.values)): 
    hist_emis.variable.values[i] = hist_emis.variable.values[i][10:]

# rename to match FaIR spceies
hist_emis.loc[hist_emis.variable == 'CO2|AFOLU', 'variable'] = 'CO2 AFOLU'
hist_emis.loc[hist_emis.variable == 'CO2|Energy and Industrial Processes', 'variable'] = 'CO2 FFI'

hist_emis.head()

,model,scenario,region,variable,unit,1750,1751,1752,1753,1754,...,2013,2014,2015,2016,2017,2018,2019,2020,2021,2022
0,Historical,GCP+CEDS+PRIMAP+GFED,World,BC,Mt BC/yr,2.096766,2.071972,2.067178,2.070382,2.098586,...,7.842259,7.934828,7.926371,7.568060,7.562319,7.459095,7.781842,6.525021,6.871947,6.701702
1,Historical,GCP+CEDS+PRIMAP+GFED,World,C2F6,kt C2F6/yr,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.003458,1.035565,0.933936,0.979945,0.996980,0.996764,1.062577,1.106550,1.160588,1.167145
2,Historical,GCP+CEDS+PRIMAP+GFED,World,C3F8,kt C3F8/yr,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.314138,0.323186,0.304124,0.319107,0.324656,0.324586,0.346019,0.360340,0.377938,0.380074
3,Historical,GCP+CEDS+PRIMAP+GFED,World,C4F10,kt C4F10/yr,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.070672,0.070641,0.067024,0.070327,0.071550,0.071536,0.076260,0.079417,0.083292,0.083759
4,Historical,GCP+CEDS+PRIMAP+GFED,World,C5F12,kt C5F12/yr,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.035588,0.031141,0.030030,0.031513,0.032065,0.032062,0.034184,0.035603,0.037338,0.037545


## Import & clean future emissions dataset
`proj_emis`

In [31]:
MSG_proj_emis = pd.read_csv('~/inputs/NGFS_MESSAGE.csv')

# remove final row- does not contain emission data
MSG_proj_emis = MSG_proj_emis.drop(index=MSG_proj_emis.index[-1])

# remove 'Region' column- not needed
MSG_proj_emis = MSG_proj_emis.drop(columns=['Region', 'Model'])

# remove 'Emissions|' from variable names
for i in range(len(MSG_proj_emis.Variable.values)): 
    MSG_proj_emis.Variable.values[i] = MSG_proj_emis.Variable.values[i][10:]
    
# rename to match FaIR spceies
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'CO2|AFOLU', 'Variable'] = 'CO2 AFOLU'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'CO2|Energy and Industrial Processes', 'Variable'] = 'CO2 FFI'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'HFC|HFC125', 'Variable'] = 'HFC-125'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'HFC|HFC134a', 'Variable'] = 'HFC-134a'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'HFC|HFC227ea', 'Variable'] = 'HFC-227ea'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'HFC|HFC143a', 'Variable'] = 'HFC-143a'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'HFC|HFC23', 'Variable'] = 'HFC-23'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'HFC|HFC245fa', 'Variable'] = 'HFC-245fa'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'HFC|HFC32', 'Variable'] = 'HFC-32'
MSG_proj_emis.loc[MSG_proj_emis.Variable == 'HFC|HFC43-10', 'Variable'] = 'HFC-4310mee'

# get rid of species not included in hist dataset (F-Gases, HFC, PFC)
MSG_proj_emis = MSG_proj_emis[~MSG_proj_emis['Variable'].isin(['F-Gases', 'HFC', 'PFC', 'CO2'])]

# remove '(version: 1)' from scenario names
for i in range(len(MSG_proj_emis.Scenario.values)): 
    MSG_proj_emis.Scenario.values[i] = MSG_proj_emis.Scenario.values[i][:-13]
    
# rename scenarios for clarity
MSG_proj_emis.loc[MSG_proj_emis.Scenario == 'Below 2?C', 'Scenario'] = 'Below 2 C'
MSG_proj_emis.loc[MSG_proj_emis.Scenario == 'Nationally Determined Contributions (NDCs)', 'Scenario'] = 'NDCs'

MSG_proj_emis.to_csv('~/outputs/NGFS_MSG_cleaned.csv', index=False)

In [32]:
proj_emis = pd.read_csv('~/inputs/NGFS_GCAM.csv')

# remove final row- does not contain emission data
proj_emis = proj_emis.drop(index=175)

# remove 'Region' column- not needed
proj_emis = proj_emis.drop(columns=['Region', 'Model'])

# remove 'Emissions|' from variable names
for i in range(len(proj_emis.Variable.values)): 
    proj_emis.Variable.values[i] = proj_emis.Variable.values[i][10:]
    
# rename to match FaIR spceies
proj_emis.loc[proj_emis.Variable == 'CO2|AFOLU', 'Variable'] = 'CO2 AFOLU'
proj_emis.loc[proj_emis.Variable == 'CO2|Energy and Industrial Processes', 'Variable'] = 'CO2 FFI'
proj_emis.loc[proj_emis.Variable == 'HFC|HFC125', 'Variable'] = 'HFC-125'
proj_emis.loc[proj_emis.Variable == 'HFC|HFC134a', 'Variable'] = 'HFC-134a'
proj_emis.loc[proj_emis.Variable == 'HFC|HFC227ea', 'Variable'] = 'HFC-227ea'
proj_emis.loc[proj_emis.Variable == 'HFC|HFC143a', 'Variable'] = 'HFC-143a'
proj_emis.loc[proj_emis.Variable == 'HFC|HFC23', 'Variable'] = 'HFC-23'
proj_emis.loc[proj_emis.Variable == 'HFC|HFC245fa', 'Variable'] = 'HFC-245fa'
proj_emis.loc[proj_emis.Variable == 'HFC|HFC32', 'Variable'] = 'HFC-32'

# get rid of species not included in hist dataset (F-Gases, HFC, PFC)
proj_emis = proj_emis[~proj_emis['Variable'].isin(['F-Gases', 'HFC', 'PFC', 'CO2'])]

# remove '(version: 1)' from scenario names
for i in range(len(proj_emis.Scenario.values)): 
    proj_emis.Scenario.values[i] = proj_emis.Scenario.values[i][:-13]
    
# rename scenarios for clarity
proj_emis.loc[proj_emis.Scenario == 'Below 2?C', 'Scenario'] = 'Below 2 C'
proj_emis.loc[proj_emis.Scenario == 'Nationally Determined Contributions (NDCs)', 'Scenario'] = 'NDCs'

proj_emis.to_csv('~/outputs/NGFS_GCAM_cleaned.csv', index=False)

Ensure that variable names in projected dataset are consistent with those of future dataset. Also identify missing species from projected dataset.

In [33]:
a = hist_emis['variable'].unique()
b = proj_emis['Variable'].unique()
c = MSG_proj_emis['Variable'].unique()

print('strings in hist but not proj:')
print('\nGCAM')
print(np.setdiff1d(a, b))
print('\nMESSAGE')
print(np.setdiff1d(a, c))

print('\nstrings in proj but not hist:')
print('\nGCAM')
print(np.setdiff1d(b, a))
print('\nMESSAGE')
print(np.setdiff1d(c, a))

strings in hist but not proj:

GCAM
['C3F8' 'C4F10' 'C5F12' 'C6F14' 'C7F16' 'C8F18' 'CCl4' 'CFC-11' 'CFC-113'
 'CFC-114' 'CFC-115' 'CFC-12' 'CH2Cl2' 'CH3Br' 'CH3CCl3' 'CH3Cl' 'CHCl3'
 'HCFC-141b' 'HCFC-142b' 'HCFC-22' 'HFC-152a' 'HFC-236fa' 'HFC-365mfc'
 'HFC-4310mee' 'Halon-1211' 'Halon-1301' 'Halon-2402' 'NF3' 'SO2F2'
 'c-C4F8']

MESSAGE
['C2F6' 'C3F8' 'C4F10' 'C5F12' 'C6F14' 'C7F16' 'C8F18' 'CCl4' 'CF4'
 'CFC-11' 'CFC-113' 'CFC-114' 'CFC-115' 'CFC-12' 'CH2Cl2' 'CH3Br'
 'CH3CCl3' 'CH3Cl' 'CHCl3' 'HCFC-141b' 'HCFC-142b' 'HCFC-22' 'HFC-152a'
 'HFC-236fa' 'HFC-365mfc' 'Halon-1211' 'Halon-1301' 'Halon-2402' 'NF3'
 'SO2F2' 'c-C4F8']

strings in proj but not hist:

GCAM
[]

MESSAGE
[]


## Plot projected vs. historical CH4 emissions

This will help us to analyze the discrepancy between the two timeseries. Then, we can use the `aneris` harmonization tool to make these trajectories consistent with one another

In [34]:
ch4_proj = proj_emis.loc[proj_emis.Variable == 'CH4'].loc[proj_emis.Scenario == 'Current Policies']
ch4_proj = ch4_proj.values[0][4:].astype(float)

In [35]:
ch4_proj

array([332.3504848, 349.4731017, 370.7043309, 383.7314283, 396.5458579,
       406.4101421, 413.584382 , 416.1188587, 418.3506344, 417.2221213,
       415.408043 , 410.5150104, 406.2395599, 401.7712512, 395.3983755,
       388.6285407])

Projections are in 5-year timesteps, so we need to interpolate over the interval

In [36]:
yrs = np.arange(2020, 2101, 5)
all_yrs = np.arange(2020, 2101)

In [37]:
ch4_proj_interp = np.interp(all_yrs, yrs, ch4_proj)

ValueError: fp and xp are not of the same length.

In [ ]:
plt.plot(np.arange(2000, 2023), hist_emis.loc[hist_emis.variable == 'CH4'].values[0][255:])
plt.plot(np.arange(2020, 2101), ch4_proj_interp)
plt.axvline(2020, linestyle='--', color='grey', alpha=0.5)
plt.xlabel('Year')
plt.ylabel('Mt CH4/yr')
plt.title('Gap Between Projected & Historical CH4 Emissions\n Current Policies Scenario')
plt.show()